## Explainer Notebook - Twitter API Scrapping

In order to scrap the needed data from the Twitter we used the list of Representatives and Senators from https://drive.google.com/file/d/1ICzekUINYn5EsTcVjgWKtAJ17qAiRp7h/view?usp=sharing.  

In [ ]:
import twitter
import csv
import pandas as pd

In [ ]:
# Secret keys to use the twitter API
consumer_key = 'YOUR_COSTUMER_KEY'
consumer_secret = 'YOUR_COSTUMER_SECRET'
access_token_key = 'YOUR_ACCESS_TOKEN_KEY'
access_token_secret = 'YOUR_ACCESS_TOKEN_SECRET'

# First, let's use the twitter API in order to scrap data
api = twitter.Api(consumer_key=consumer_key,
                  consumer_secret=consumer_secret,
                  access_token_key=access_token_key,
                  access_token_secret=access_token_secret)

# From the list we obtain the `Party`, `TwitterUsernames`, `Name`, etc..
with open('./data/2020 representatives list.csv', newline='', encoding="utf8") as f:
    reader = csv.reader(f)
    data = list(reader)

    Party = []
    TwitterUsernames = []
    Names = []
    State = []
    Chamber = []

    for line in data:
        if line[4] != "":
            TwitterUsernames.append(line[4].replace("@", ""))
            Names.append(line[2])
            Chamber.append(line[1])
            State.append(line[0])

            if line[3] == "R":
                Party.append("Republican")
            else:
                Party.append("Democrat")

# Let's create the DataFrame
Data = pd.DataFrame()

# the 1st row is the names of the list
Party.pop(0), State.pop(0), Chamber.pop(0), Names.pop(0), TwitterUsernames.pop(0)  # drop the index

Data['Party'] = Party
Data['Name'] = Names
Data['State'] = State
Data['Chamber'] = Chamber
Data['TwitterUsernames'] = TwitterUsernames

Timelines1 = []
for name in TwitterUsernames[0:100]:
    Timelines1.append(api.GetUserTimeline(screen_name=name, count=200))  # 200 tweets

Timelines2 = []
for name in TwitterUsernames[100:200]:
    Timelines2.append(api.GetUserTimeline(screen_name=name, count=200))

Timelines3 = []
for name in TwitterUsernames[200:300]:
    Timelines3.append(api.GetUserTimeline(screen_name=name, count=200))

Timelines4 = []
for name in TwitterUsernames[300:400]:
    Timelines4.append(api.GetUserTimeline(screen_name=name, count=200))

Timelines5 = []
for name in TwitterUsernames[400:]:
    Timelines5.append(api.GetUserTimeline(screen_name=name, count=200))

# combining all the timelines together, form on dataset w tweets
Timelines = []
Timelines = Timelines1 + Timelines2 + Timelines3 + Timelines4 + Timelines5

# Let's grab all the content of the tweets into the `ListOftxt` list.

ListOftxt = []

for timeline in Timelines:
    fullTxt = ''

    for tweet in timeline:
        fullTxt += tweet.text

    ListOftxt.append(fullTxt)

Data['Tweets'] = ListOftxt

# save dataframe to csv file
Data.to_csv(r'./data/Data.csv', index=False)